#### Problem 5, Part 2: Multiagent Bidding Strategy Leaderboard and a new model

In [2]:
import pandas as pd

#only validation set as well
comp_df = pd.read_csv('validationcompetition.csv')    

comp_df.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,bidprice,payprice,keypage,advertiser,usertag,lgrlinear,xgblinear,xgbsquare,xgbortb1,xgbortb2
0,0,4,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,windows_ie,211.144.203.*,79,79,1.0,...,300,23,bebefa5efe83beee17a3d245e7c5085b,1458,"1,386,610,111",50.133784,28.3046,11.5120,46.1898,52.2547
1,0,1,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,windows_chrome,58.247.250.*,79,79,1.0,...,238,75,a3becb967c015e9acb5e72f22b100ef8,3476,"1,007,510,057,100,240,000,000,000,000,000,000,...",20.858784,1.8620,4.9820,37.4624,4.5445
2,0,4,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,windows_ie,117.12.111.*,2,2,2.0,...,238,65,d29e59bf0f7f8243858b8183f14d4412,3358,"1,386,610,024,100,590,000,000,000,000,000,000,...",28.953844,46.6081,31.2147,5.9274,61.7072
3,0,5,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,windows_chrome,61.187.224.*,201,205,2.0,...,238,6,d29e59bf0f7f8243858b8183f14d4412,3358,"1,386,610,057,100,060,000,000,000",26.981120,27.3139,10.7202,45.3741,5.1638
4,0,1,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,windows_chrome,117.41.145.*,134,135,2.0,...,249,5,7e0e909ea89f707bc20f9585c9072081,3476,"10,006,100,631,377,600,000",31.311401,20.2074,58.6756,3.9027,4.6702


In [3]:
comp_df.shape

(303925, 30)

In [8]:
#%%pixie_debugger

#simple procedure to see all strategies in one leaderboard. We used indexes and values in arrays because it is faster to run

import numpy as np

np.set_printoptions(suppress=True)

leaderboard = np.empty((5,5), dtype=object) 
temparray = np.empty((5,2), dtype=object)

#---

budget_agent = [6250000,6250000,6250000,6250000,6250000]
clicks = [0,0,0,0,0]
impressions = [0,0,0,0,0]

#---

for i in range(0, len(comp_df)):  

    #new temporary array with values from each row
    temparray[0, 0], temparray[0, 1] = 0, comp_df.lgrlinear[i]
    temparray[1, 0], temparray[1, 1] = 1, comp_df.xgblinear[i]
    temparray[2, 0], temparray[2, 1] = 2, comp_df.xgbsquare[i]
    temparray[3, 0], temparray[3, 1] = 3, comp_df.xgbortb1[i]
    temparray[4, 0], temparray[4, 1] = 4, comp_df.xgbortb2[i]

    temparray = temparray[temparray[:,1].argsort()]  #sort by value
    
    maxindex = temparray[0,0]   #max index from sorted the array
    maxvalue = temparray[0,1]   #max value from sorted the array
    secindex = temparray[1,0]   #second max index from the sorted array
    secvalue = temparray[1,1]   #second max value from the sorted array

    #comparison
    if budget_agent[maxindex] > maxvalue:

        impressions[maxindex] += 1
        budget_agent[maxindex] = budget_agent[maxindex] - secvalue    
        clicks[maxindex] = clicks[maxindex] + comp_df.click[i]
        
    else:
        break
        
#all results in resulting array, just for convinience

for x in range(0,5):
    
    leaderboard[x,0] = 'Group' + str(x+1)
    leaderboard[x,1] = clicks[x]
    leaderboard[x,2] = impressions[x]
    leaderboard[x,3] = round(clicks[x]/impressions[x],6)
    leaderboard[x,4] = round(budget_agent[x]/1000,2)

In [18]:
#results sorted by number of clicks

leaderboard[leaderboard[:,-1].argsort()]

array([['Group3', 71, 81319, 0.000873, 4863.48],
       ['Group5', 54, 83607, 0.000646, 5176.44],
       ['Group2', 44, 66341, 0.000663, 5434.09],
       ['Group4', 30, 64058, 0.000468, 5447.63],
       ['Group1', 3, 8600, 0.000349, 6071.88]], dtype=object)

Group 1 = Logistic regression Linear bidding strategy 

Group 2 = XGBoost Linear Bidding Strategy

Group 3 = XGBoost Square Bidding Strategy

Group 4 = XGBoost ORTB1 Strategy

Group 5 = XGBoost ORTB2 Strategy